# 1 Object

* object is a Real-world-entity.
* Anything which is persent in the real world  and has some 
* 


```TS
import { HttpClient } from '@angular/common/http';
import {
  ChangeDetectorRef,
  Component,
  EventEmitter,
  Input,
  OnInit,
  Output,
  ViewChild,
} from '@angular/core';
import { UntypedFormControl } from '@angular/forms';
import { GoogleMap, MapInfoWindow } from '@angular/google-maps';
import { MatAutocompleteActivatedEvent } from '@angular/material/autocomplete';
import { BehaviorSubject, catchError, map, Observable, of } from 'rxjs';
import { TranslationService } from 'src/app/services/translation.service';
import { VCAGoogleAddress } from 'src/app/utils/interface-and-classes/vca-google-address-class';
import { environment } from 'src/environments/environment';

@Component({
  selector: 'google-map-address-picker',
  templateUrl: './google-map-address-picker.component.html',
  styleUrls: ['./google-map-address-picker.component.scss'],
})
export class GoogleMapAddressPickerComponent implements OnInit {
  map = '';
  @Input() latlng?: BehaviorSubject<any>;
  @ViewChild('map', { static: false }) mapRef!: GoogleMap;
  @Output() addressPicked: EventEmitter<VCAGoogleAddress> = new EventEmitter();
  @ViewChild(MapInfoWindow) infoWindow!: MapInfoWindow;
  zoom = 14;
  center!: google.maps.LatLngLiteral;
  options: google.maps.MapOptions = {
    zoomControl: true,
    scrollwheel: false,
    disableDoubleClickZoom: true,
    maxZoom: 20,
    minZoom: 4,
  };
  markerOptions: google.maps.MarkerOptions = { draggable: false };
  markerPositions: google.maps.LatLngLiteral[] = [];
  autocompleteService!: google.maps.places.AutocompleteService;
  predictions?: google.maps.places.AutocompletePrediction[] | null;
  address!: VCAGoogleAddress;
  searchInput: UntypedFormControl = new UntypedFormControl();
  car = '';
  translationVar: any;
  apiLoaded: Observable<boolean>;

  constructor(
    private cdRef: ChangeDetectorRef,
    private translation: TranslationService,
    httpClient: HttpClient
  ) {
    const url =
      'https://maps.googleapis.com/maps/api/js?key=' +
      environment.googleMapsKey +
      '&libraries=places';
    this.apiLoaded = httpClient.jsonp(url, 'callback').pipe(
      map(() => {
        this.autocompleteService = new google.maps.places.AutocompleteService();
        return true;
      }),
      catchError((err) => {
        console.log(err);
        return of(false);
      })
    );
  }

  ngOnInit() {
    this.translationVar = this.translation.translationVar.addressBook;

    // this.autocompleteService = new google.maps.places.AutocompleteService();
    if ("geolocation" in navigator) {
      navigator.geolocation.getCurrentPosition(
        (position) => {
          if (position && !this.center) {
            this.center = {
              lat: position.coords.latitude,
              lng: position.coords.longitude
            }
          }
        },
        () => {
          if (!this.center) {
            this.center = {
              lat: 12.9716,
              lng: 77.5946,
            };
          }
        }
      );
    }
    if (this.latlng) {
      this.latlng.subscribe((data) => {
        if (data.lat) {
          this.center = {
            lat: data.lat,
            lng: data.lng,
          };
          this.markerPositions[0] = this.center;
          this.zoom = 19;
          this.cdRef.detectChanges();
        }
      });
    }
  }
  addMarker(event: any) {
    this.markerPositions[0] = event.latLng!.toJSON();
    this.findAddress(event);
  }


  findAddress(event:any) {
    this.address = new VCAGoogleAddress();
    let place: any;
    const placeDetail = new google.maps.places.PlacesService(
      this.mapRef!.googleMap!
    );
    placeDetail.getDetails(
      {
        fields: ['address_components', 'geometry', 'name'],
        placeId: event.placeId,
      },
      (addressDetails, status) =>{
        console.log("getAddressDetails  :-  " , addressDetails , "    status :-   " , status);

        if (status === google.maps.places.PlacesServiceStatus.OK) {
          place = addressDetails!.address_components;
          this.address.name = addressDetails!.name!;
        }

        this.address.googleAddressId = event.placeId;
        this.setAddressDetails(place);
        // for (let i = 0; i < place.length; i++) {
        //   if (place[i].types[0] === 'street_number') {
        //     this.address.addressLine1 = place[i].long_name;
        //   }
        //   if (place[i].types[0] === 'route') {
        //     if (!this.address.addressLine1) {
        //       this.address.addressLine1 = place[i].long_name;
        //     } else {
        //       this.address.addressLine1 += ' , ' + place[i].long_name;
        //     }
        //   } else if (/sublocality_level/gi.test(place[i].types[0])) {
        //     if (!this.address.addressLine1) {
        //       this.address.addressLine1 = place[i].long_name;
        //     } else if (!this.address.addressLine2) {
        //       this.address.addressLine2 = place[i].long_name;
        //     } else if (!this.address.addressLine3) {
        //       this.address.addressLine3 = place[i].long_name;
        //     }
        //   } else if (
        //     place[i].types[0] === 'locality' &&
        //     !this.address.addressLine3
        //   ) {
        //     this.address.addressLine3 = place[i].long_name;
        //   } else if (place[i].types[0] === 'administrative_area_level_2') {
        //     this.address.city = place[i].long_name;
        //   } else if (place[i].types[0] === 'administrative_area_level_1') {
        //     this.address.state = place[i].long_name;
        //   } else if (place[i].types[0] === 'country') {
        //     this.address.country = place[i].long_name;
        //   } else if (place[i].types[0] === 'postal_code') {
        //     this.address.zip = place[i].long_name;
        //   }
        // }
        // this.addressPicked.emit(this.address);
      })


}

setAddressDetails(place:any){
      for (let i = 0; i < place.length; i++) {
        if (place[i].types[0] === 'street_number') {
          this.address.addressLine1 = place[i].long_name;
        }
        if (place[i].types[0] === 'route') {
          if (!this.address.addressLine1) {
            this.address.addressLine1 = place[i].long_name;
          } else {
            this.address.addressLine1 += ' , ' + place[i].long_name;
          }
        } else if (/sublocality_level/gi.test(place[i].types[0])) {
          if (!this.address.addressLine1) {
            this.address.addressLine1 = place[i].long_name;
          } else if (!this.address.addressLine2) {
            this.address.addressLine2 = place[i].long_name;
          } else if (!this.address.addressLine3) {
            this.address.addressLine3 = place[i].long_name;
          }
        } else if (
          place[i].types[0] === 'locality' &&
          !this.address.addressLine3
        ) {
          this.address.addressLine3 = place[i].long_name;
        } else if (place[i].types[0] === 'administrative_area_level_2') {
          this.address.city = place[i].long_name;
        } else if (place[i].types[0] === 'administrative_area_level_1') {
          this.address.state = place[i].long_name;
        } else if (place[i].types[0] === 'country') {
          this.address.country = place[i].long_name;
        } else if (place[i].types[0] === 'postal_code') {
          this.address.zip = place[i].long_name;
        }
      }
      this.addressPicked.emit(this.address);
}

  getAddress() {
    this.autocompleteService.getPlacePredictions(
      { input: this.searchInput.value },
      (predictions, status) => {
        if (status !== google.maps.places.PlacesServiceStatus.OK) {
          alert(this.searchInput.value + ' not found');
          return;
        }
        this.predictions = predictions;
      }
    );
  }
  getAddressDetails(address: any) {
    this.address = new VCAGoogleAddress();
    let place: any;
    const placeDetail = new google.maps.places.PlacesService(
      this.mapRef!.googleMap!
    );
    placeDetail.getDetails(
      {
        fields: ['address_components', 'geometry', 'name'],
        placeId: address.place_id,
      },
      (addressDetails, status) => {
        if (status === google.maps.places.PlacesServiceStatus.OK) {
          place = addressDetails!.address_components;
          this.address.name = addressDetails!.name!;
          const lat = addressDetails!.geometry!.location!.lat();
          const lng = addressDetails!.geometry!.location!.lng();
          this.address.latitude = lat.toString();
          this.address.longitude = lng.toString();

          this.center = {
            lat: lat,
            lng: lng,
          };
          this.zoom = 19;
          this.markerPositions[0] = this.center;
          this.cdRef.detectChanges();
        }
        this.address.googleAddressId = address.place_id;
        this.address.description = address.description;
        this.setAddressDetails(place);

        // for (let i = 0; i < place.length; i++) {
        //   if (place[i].types[0] === 'street_number') {
        //     this.address.addressLine1 = place[i].long_name;
        //   }
        //   if (place[i].types[0] === 'route') {
        //     if (!this.address.addressLine1) {
        //       this.address.addressLine1 = place[i].long_name;
        //     } else {
        //       this.address.addressLine1 += ' , ' + place[i].long_name;
        //     }
        //   } else if (/sublocality_level/gi.test(place[i].types[0])) {
        //     if (!this.address.addressLine1) {
        //       this.address.addressLine1 = place[i].long_name;
        //     } else if (!this.address.addressLine2) {
        //       this.address.addressLine2 = place[i].long_name;
        //     } else if (!this.address.addressLine3) {
        //       this.address.addressLine3 = place[i].long_name;
        //     }
        //   } else if (
        //     place[i].types[0] === 'locality' &&
        //     !this.address.addressLine3
        //   ) {
        //     this.address.addressLine3 = place[i].long_name;
        //   } else if (place[i].types[0] === 'administrative_area_level_2') {
        //     this.address.city = place[i].long_name;
        //   } else if (place[i].types[0] === 'administrative_area_level_1') {
        //     this.address.state = place[i].long_name;
        //   } else if (place[i].types[0] === 'country') {
        //     this.address.country = place[i].long_name;
        //   } else if (place[i].types[0] === 'postal_code') {
        //     this.address.zip = place[i].long_name;
        //   }
        // }
        // this.addressPicked.emit(this.address);
      }
    );
  }



  optionSelected(evt: MatAutocompleteActivatedEvent) {
    this.getAddressDetails(evt.option?.value);
    console.log("this.evt.option?.value" , evt.option?.value)
  }

  getOptionText(option: any) {
    return option ? option.description : null;
  }
}

```